In [1]:
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
import pickle
import mlflow

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1722989988527, experiment_id='1', last_update_time=1722989988527, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [5]:
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [8]:
reg = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.8, n_estimators=500, verbosity=2)
reg.fit(X_train, y_train)

y_pred = reg.predict(X_val)

root_mean_squared_error(y_val, y_pred)

6.429234961685116

In [15]:
with open('models/xgb_reg.bin', 'wb') as f_out:
    pickle.dump((dv, reg), f_out)

In [11]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ivan")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("validation-data-path", "./data/green_tripdata_2021-02.parquet")

    lr = 0.07
    mlflow.log_param("learning rate", lr)
    reg = xgb.XGBRegressor(learning_rate=lr)
    reg.fit(X_train, y_train)

    y_pred = reg.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/xgb_reg.bin", artifact_path="models_pickle")

In [13]:
from hyperopt import fmin,tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [9]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [13]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [17]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:19:31] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.57254                           
[1]	validation-rmse:8.15383                           
[2]	validation-rmse:7.43418                           
[3]	validation-rmse:7.07425                           
[4]	validation-rmse:6.89460                           
[5]	validation-rmse:6.79593                           
[6]	validation-rmse:6.74241                           
[7]	validation-rmse:6.70897                           
[8]	validation-rmse:6.68691                           
[9]	validation-rmse:6.66982                           
[10]	validation-rmse:6.65470                          
[11]	validation-rmse:6.64547                          
[12]	validation-rmse:6.64017                          
[13]	validation-rmse:6.63559                          
[14]	validation-rmse:6.63130                          
[15]	validation-rmse:6.62968                          
[16]	validation-rmse:6.62674                          
[17]	validation-rmse:6.62564                          
[18]	valid

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:21:06] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.02429                                                      
[1]	validation-rmse:7.03393                                                      
[2]	validation-rmse:6.80174                                                      
[3]	validation-rmse:6.72467                                                      
[4]	validation-rmse:6.69884                                                      
[5]	validation-rmse:6.67900                                                      
[6]	validation-rmse:6.67120                                                      
[7]	validation-rmse:6.66856                                                      
[8]	validation-rmse:6.66456                                                      
[9]	validation-rmse:6.66269                                                      
[10]	validation-rmse:6.65935                                                     
[11]	validation-rmse:6.65582                                                     
[12]	validation-

...

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:26:12] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.14217                                                   
[1]	validation-rmse:10.25662                                                   
[2]	validation-rmse:9.52919                                                    
[3]	validation-rmse:8.93678                                                    
[4]	validation-rmse:8.45678                                                    
[5]	validation-rmse:8.07076                                                    
[6]	validation-rmse:7.76131                                                    
[7]	validation-rmse:7.51502                                                    
[8]	validation-rmse:7.31778                                                    
[9]	validation-rmse:7.16185                                                    
[10]	validation-rmse:7.03760                                                   
[11]	validation-rmse:6.93908                                                   
[12]	validation-rmse:6.85913            

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [00:29:18] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.33025                                                       
[1]	validation-rmse:6.75300                                                       
[2]	validation-rmse:6.64756                                                       
[3]	validation-rmse:6.61726                                                       
[4]	validation-rmse:6.60720                                                       
[5]	validation-rmse:6.59708                                                       
[6]	validation-rmse:6.58898                                                       
[7]	validation-rmse:6.58376                                                       
[8]	validation-rmse:6.57365                                                       
[9]	validation-rmse:6.56807                                                       
[10]	validation-rmse:6.56318                                                      
[11]	validation-rmse:6.55864                                                      
[12]

KeyboardInterrupt: 

In [19]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2024/08/10 00:33:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


KeyboardInterrupt: 

In [16]:
with mlflow.start_run():
    best_params= {'learning_rate': 0.33166586938408643,
             'max_depth': 10,
             'min_child_weight': 2.968412805214042,
             'objective': 'reg:linear',
             'reg_alpha': 0.037155048659385764,
             'reg_lambda': 0.21411947066104986}

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:11:55] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.71820
[1]	validation-rmse:8.31407
[2]	validation-rmse:7.56075
[3]	validation-rmse:7.15731
[4]	validation-rmse:6.94778
[5]	validation-rmse:6.83616
[6]	validation-rmse:6.77306
[7]	validation-rmse:6.73217
[8]	validation-rmse:6.70788
[9]	validation-rmse:6.69153
[10]	validation-rmse:6.68157
[11]	validation-rmse:6.67640
[12]	validation-rmse:6.66892
[13]	validation-rmse:6.66567
[14]	validation-rmse:6.66253
[15]	validation-rmse:6.65949
[16]	validation-rmse:6.65684
[17]	validation-rmse:6.65345
[18]	validation-rmse:6.65014
[19]	validation-rmse:6.64767
[20]	validation-rmse:6.64427
[21]	validation-rmse:6.64173
[22]	validation-rmse:6.63861
[23]	validation-rmse:6.63483
[24]	validation-rmse:6.63196
[25]	validation-rmse:6.63124
[26]	validation-rmse:6.62895
[27]	validation-rmse:6.62590
[28]	validation-rmse:6.62410
[29]	validation-rmse:6.62273
[30]	validation-rmse:6.62121
[31]	validation-rmse:6.61964
[32]	validation-rmse:6.61700
[33]	validation-rmse:6.61555
[34]	validation-rmse:6.6

/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [01:13:10] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an alrea

In [10]:
import mlflow
logged_model = 'runs:/ea0b5d61e0d14ef3973d8dc436e2c061/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [12]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [13]:
y_pred = xgboost_model.predict(valid)

In [14]:
y_pred[:10]

array([14.379823 ,  7.0384655, 16.307316 , 24.68176  ,  9.488231 ,
       17.321255 , 11.336303 ,  7.744595 ,  9.077354 , 19.34526  ],
      dtype=float32)